In [437]:
import pandas as pd
import os,sys, imp, glob
sys.path.append('./src/')
import doc_utils_clean as doc_utils
import pos_yap_process
import feature_utils
import common_utils
import defines
import numpy as np
import re

In [438]:
doc_list =  glob.glob(os.path.join(os.path.join(os.getcwd(),"tmp","agreement",r"*_Compare_*.docx")))
doc_list.sort()
doc_map = {}

for idx,doc in enumerate(doc_list):
    f_name=re.sub('.docx','',os.path.basename(doc).split('_')[-1])
    doc_map[f_name] = doc
doc_map

{'Dana': '/scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/tmp/agreement/01_Compare_Dana.docx',
 'Mor': '/scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/tmp/agreement/02_Compare_Mor.docx'}

In [439]:
dir_name="agreement"

In [440]:
doc_db=pd.read_csv(os.path.join(os.getcwd(), defines.PATH_TO_DFS, dir_name, "doc_db.csv"))
doc_db

,path,file_name,doc_idx_from_name
0,/scratch/sim_reg6/users/zsofya/classroom/MSc/p...,01_Compare_Dana.docx,1
1,/scratch/sim_reg6/users/zsofya/classroom/MSc/p...,02_Compare_Mor.docx,2


In [441]:
dir_name

'agreement'

In [442]:
imp.reload(doc_utils)
doc_utils.parse_all_docs(dir_name=dir_name,merge_short_sent=False,doc_path_list=doc_list)

Creating doc_db
738 sentences 0 738 sentences 1 

In [443]:
db = {}
for db_type in ['par_db','sent_db','block_db']:
    db[db_type]={}
    for idx in doc_db['doc_idx_from_name'].values:
        db[db_type][idx]=pd.read_csv(os.path.join(os.getcwd(), defines.PATH_TO_DFS, dir_name, "{:02d}_{}.csv".format(idx,db_type)))

In [444]:
for key,value in db.items():
    print(key,len(value[1]),len(value[2]))


par_db 164 164
sent_db 738 738
block_db 212 200


In [445]:
sent_count={}
sent_count[1]=db['sent_db'][1].groupby('par_idx_in_doc').size()

In [446]:
sent_count[2]=db['sent_db'][2].groupby('par_idx_in_doc').size()

In [447]:
sent_count[2]

par_idx_in_doc
0.000000      1
1.000000      1
2.000000      2
3.000000      1
4.000000      1
             ..
159.000000    1
160.000000    2
161.000000    1
162.000000    5
163.000000    8
Length: 160, dtype: int64

In [448]:
db_diff = pd.DataFrame()
db_diff['1']=sent_count[1]
db_diff['2']=sent_count[2]
db_diff

,1,2
par_idx_in_doc,,
0.000000,1,1
1.000000,1,1
2.000000,2,2
3.000000,1,1
4.000000,1,1
...,...,...
159.000000,1,1
160.000000,2,2
161.000000,1,1


In [449]:
db_diff[db_diff['1']!=db_diff['2']]

,1,2
par_idx_in_doc,,


In [450]:
diff_par_indices=db_diff[db_diff['1']!=db_diff['2']].index

In [451]:
with pd.option_context('display.max_colwidth', None):
    display(db['block_db'][1].query("par_db_idx.isin(@diff_par_indices)")[['text','par_db_idx','par_type','is_nar']])

,text,par_db_idx,par_type,is_nar


In [452]:
with pd.option_context('display.max_colwidth', None):
    display(db['block_db'][2].query("par_db_idx.isin(@diff_par_indices)")[['text','par_db_idx','par_type','is_nar']])

,text,par_db_idx,par_type,is_nar


In [453]:
for idx in diff_par_indices:
    with pd.option_context('display.max_colwidth', None):
        display(db['sent_db'][1].query("par_idx_in_doc == @idx")[['text','sent_idx_in_par','par_type','is_nar','par_idx_in_doc']])

In [454]:
for idx in diff_par_indices:
    with pd.option_context('display.max_colwidth', None):
        display(db['sent_db'][2].query("par_idx_in_doc == @idx")[['text','sent_idx_in_par','par_type','is_nar','par_idx_in_doc']])

In [ ]:
for idx in diff_par_indices:
    with pd.option_context('display.max_colwidth', None):
        display(db['sent_db'][1].query("par_idx_in_doc == @idx")[['text','sent_idx_in_par','par_type','is_nar','par_idx_in_doc']])